# basic file clean up

In [1]:
import sys
sys.path.append('../../')
import glob
import re

import pandas as pd

from config import CLEAN_DATA_DIR, OUTPUT_DIR, RAW_DATA_DIR

from scripts.normalize_data import (
    csv_cleanup,
    update_metadata,
    delete_duplicate_columns,
    change_file_encoding,
    remove_whitespace,
    check_duplicate_columns
)

In [32]:
clean_data_path = CLEAN_DATA_DIR

metadata_file = OUTPUT_DIR/'metadata'/'LIMS'/'Micropal_changes.csv' 
# metadata_file = OUTPUT_DIR/'metadata'/'LIMS'/'Micropal_changes_4.csv' 


## fix bad encoding

In [3]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_group,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,update_sample_col,update_top_bottom,add_missing_cols
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,True,True,False,True,False,False,False,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,True,False,False,False,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,False,True,False,False,True,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,True,False,False,True,False,False


In [4]:
def process_file(path):
    with open(clean_data_path/path, "r") as file:
        changed = False
        try:
            filedata = file.read()
            
        except UnicodeDecodeError as err:
            change_file_encoding(clean_data_path/path)
            changed = True
    return changed
            
change_columns = [process_file(path) for path in metadata['path']]

In [5]:
dict = {"change_file_encoding": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,path,taxon_group,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,update_sample_col,update_top_bottom,add_missing_cols
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,True,True,False,True,False,False,False,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,True,False,False,False,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,False,True,False,False,True,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,True,False,False,True,False,False


In [6]:
new_metadata.to_csv(metadata_file, index=False)

## remove bad characters

In [7]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_group,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,update_sample_col,update_top_bottom,add_missing_cols
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,True,True,False,True,False,False,False,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,True,False,False,False,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,False,True,False,False,True,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,True,False,False,True,False,False


In [8]:
def clean_up_file(path):
    change = False
    with open(clean_data_path/path, "r", errors='ignore') as file:
        filedata = file.read()

        new_filedata = filedata
        new_filedata = new_filedata.replace('�m', 'µm')
        new_filedata = new_filedata.replace('æm', 'µm')

    if new_filedata != filedata:
        with open(clean_data_path/path, "w") as file:
            file.write(new_filedata)
            change = True
            
    return change
            
change_columns = [clean_up_file(file) for file in metadata['path']]


In [9]:
dict = {"remove_bad_characters": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,path,taxon_group,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,update_sample_col,update_top_bottom,add_missing_cols
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,True,True,False,True,False,False,False,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,True,False,False,False,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,False,True,False,False,True,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,True,False,False,True,False,False


In [10]:
new_metadata.to_csv(metadata_file, index=False)

## open and save file

In [11]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_group,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,update_sample_col,update_top_bottom,add_missing_cols
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,True,True,False,True,False,False,False,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,True,False,False,False,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,False,True,False,False,True,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,True,False,False,True,False,False


In [12]:
def process_file(file):
    path = f"{clean_data_path}/{file}"
    df = pd.read_csv(path, dtype=str) 
    df = csv_cleanup(df, path)
    df.to_csv(path, index=False)
            
change_columns = [process_file(file) for file in metadata['path']]


## remove empty rows

In [13]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_group,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,update_sample_col,update_top_bottom,add_missing_cols
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,True,True,False,True,False,False,False,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,True,False,False,False,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,False,True,False,False,True,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,True,False,False,True,False,False


In [14]:
def process_file(file):
    path = clean_data_path/file
    df = pd.read_csv(path, dtype=str)
    count = len(df)
    
    df.dropna(how='all', axis='rows', inplace=True)
    
    new_count = len(df)        
    changed = count != new_count
    
    if changed:
        df = csv_cleanup(df, path)
        df.to_csv(path, index=False)
        
    return changed
            
change_columns = [process_file(path) for path in metadata['path']]

In [15]:
dict = {"remove_empty_rows": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,path,taxon_group,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,update_sample_col,update_top_bottom,add_missing_cols
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,True,True,False,True,False,False,False,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,True,False,False,False,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,False,True,False,False,True,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,True,False,False,True,False,False


In [16]:
new_metadata.to_csv(metadata_file, index=False)

## remove leading/trailing spaces

In [17]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_group,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,update_sample_col,update_top_bottom,add_missing_cols
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,True,True,False,True,False,False,False,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,True,False,False,False,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,False,True,False,False,True,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,True,False,False,True,False,False


In [18]:
def process_file(file):
    path = f"{clean_data_path}/{file}"
    df = pd.read_csv(path, dtype=str, header=None)
    dup_df = df.copy()
    
    remove_whitespace(df)
    
    changed = not df.equals(dup_df)
    if changed:
        df.to_csv(path, index=False, header=False)

    return changed
            
change_columns = [process_file(file) for file in metadata['path']]


In [19]:
dict = {"remove_spaces": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,path,taxon_group,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,update_sample_col,update_top_bottom,add_missing_cols
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,True,True,False,True,False,False,False,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,True,False,False,False,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,False,True,False,False,True,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,True,False,False,True,False,False


In [20]:
new_metadata.to_csv(metadata_file, index=False)

## remove duplicate rows

In [21]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_group,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,update_sample_col,update_top_bottom,add_missing_cols
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,True,True,False,True,False,False,False,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,True,False,False,False,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,False,True,False,False,True,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,True,False,False,True,False,False


In [22]:
def delete_duplicate_rows(file):
    path = f"{clean_data_path}/{file}"
    df = pd.read_csv(path, dtype=str)
    
    columns = df.columns
    df = df.drop_duplicates()
    new_columns = df.columns
    
    changed = len(columns) != len(new_columns)
    
    if changed:
        df = csv_cleanup(df, path)
        df.to_csv(path, index=False)

    return changed
    
change_columns = [delete_duplicate_rows(file) for file in metadata['path']]


In [23]:
dict = {"delete_duplicate_rows": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,path,taxon_group,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,update_sample_col,update_top_bottom,add_missing_cols
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,True,True,False,True,False,False,False,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,True,False,False,False,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,False,True,False,False,True,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,True,False,False,True,False,False


In [24]:
new_metadata.to_csv(metadata_file, index=False)

## remove renamed duplicate column names with same values

In [25]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_group,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,update_sample_col,update_top_bottom,add_missing_cols
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,True,True,False,True,False,False,False,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,True,False,False,False,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,False,True,False,False,True,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,True,False,False,True,False,False


In [26]:
def delete_columns(file):
    path = f"{clean_data_path}/{file}"
    df = pd.read_csv(path, dtype=str)
    
    columns = df.columns
    delete_duplicate_columns(df)
    new_columns = df.columns
    
    changed = len(columns) != len(new_columns)

    if changed:
        df = csv_cleanup(df, path)
        df.to_csv(path, index=False)

    return changed
    
change_columns = [delete_columns(file) for file in metadata['path']]


In [27]:
dict = {"delete_duplicate_columns": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,path,taxon_group,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,update_sample_col,update_top_bottom,add_missing_cols
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,True,True,False,True,False,False,False,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,True,False,False,False,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,False,True,False,False,True,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,True,False,False,True,False,False


In [28]:
new_metadata.to_csv(metadata_file, index=False)

## duplicate column names with same values

In [33]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_groups,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,add_expedition_section_cols,update_sample_col,update_top_bottom,add_missing_cols,clean_up_taxa_values,clean_up_taxa_metadata_values,split_dextral_sinistral
0,363-U1482A-Benthic_Forams.csv,LIMS/Micropal_CSV_1/363-U1482A-Benthic_Forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False,False,False,True,False
1,320_U1336A_Nannofossils_2.csv,LIMS/Micropal_CSV_1/320_U1336A_Nannofossils_2.csv,nannofossils,False,False,False,True,False,False,False,True,False,False,False,False,False
2,375_U1518F_planktic_forams.csv,LIMS/Micropal_CSV_1/375_U1518F_planktic_forams...,planktic_forams,False,False,False,True,False,False,True,False,False,False,False,False,False
3,320_U1334A_Nannofossils_1.csv,LIMS/Micropal_CSV_1/320_U1334A_Nannofossils_1.csv,nannofossils,False,False,False,True,False,False,False,True,False,False,False,False,False
4,318_U1358B_Palynology.csv,LIMS/Micropal_CSV_1/318_U1358B_Palynology.csv,palynology,False,False,False,False,False,False,False,True,False,False,False,False,False


In [34]:
bad_files = []

def duplicate_columns(file):
    path = clean_data_path/file
    content = pd.read_csv(path)
    content.dropna(inplace=True, axis=1, how='all')

    res = check_duplicate_columns(content, path)
    if res:
        bad_files.append(res)

for file in metadata['path']:
    duplicate_columns(file) 


In [35]:
bad_files

[]